In [1]:
import os


In [2]:
%pwd

'/home/armando-albornoz/Desktop/ml/MLOPS_course/project1/datascienceendtoend1/research'

In [3]:
os.chdir("../")
%pwd

'/home/armando-albornoz/Desktop/ml/MLOPS_course/project1/datascienceendtoend1'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    """Configuration class for data ingestion parameters."""
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [5]:
from src.datascience.constants import * 
from src.datascience.utils.common import read_yaml, create_directories
from src.datascience import logger


[2025-08-01 21:47:06,591: INFO: __init__: Logger initialized for the datascience package.]


In [6]:
class ConfigurationManager:
    """
    Configuration manager for handling YAML configuration files.
    
    This class loads configuration, parameters, and schema files and provides
    methods to retrieve specific configuration objects.
    """
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath= PARAMS_FILE_PATH, schema_filepath = SCHEMA_FILE_PATH ):

        """
        Initialize the ConfigurationManager.
        
        Args:
            config_filepath (Path): Path to the main configuration file
            params_filepath (Path): Path to the parameters file
            schema_filepath (Path): Path to the schema file
        """
          
        try:
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)
            self.schema = read_yaml(schema_filepath)
            
            # Create artifacts root directory
            create_directories([self.config.artifacts_root])
            logger.info("ConfigurationManager initialized successfully")
            
        except Exception as e:
            logger.error(f"Error initializing ConfigurationManager: {e}")
            raise


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config

In [7]:
## Component-Data Ingestion
import urllib.request as request
import zipfile

class DataIngestion:
    """
    Component for data ingestion operations including downloading and extracting files.
    """
    def __init__(self, config:DataIngestionConfig):

        """
        Initialize DataIngestion with configuration.
        
        Args:
            config (DataIngestionConfig): Configuration object containing data ingestion parameters
        """
             
        self.config = config
    
    def download_file(self) -> None:
        """
        Download file from source URL if it doesn't already exist locally.
        
        Raises:
            Exception: If there's an error downloading the file
        """
        try:
            if not os.path.exists(self.config.local_data_file):
                # Ensure parent directory exists before downloading
                os.makedirs(os.path.dirname(self.config.local_data_file), exist_ok=True)
                
                filename, headers = request.urlretrieve(
                    url=self.config.source_URL,
                    filename=self.config.local_data_file
                )
                logger.info(f"{filename} download completed!")
            else:
                logger.info(f"File {self.config.local_data_file} already exists!")
                
        except Exception as e:
            logger.error(f"Error downloading file from {self.config.source_URL}: {e}")
            raise


    def extract_zip_file(self):

        """
        Exctract ZIP file to the specified directory.

        Raises:
            FileNotFoundError: If the ZIP file doesn't exist
            zipfile.BadZipFile: If the file is not a valid ZIP file
            Exception: If there's an error extracting the file
        """

        try:
            if not os.path.exists(self.config.local_data_file):
                raise FileNotFoundError(f"ZIP file not found {self.config.local_data_file}")
            
            unzip_path = self.config.unzip_dir
            os.makedirs(unzip_path, exist_ok=True)


            with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)

            logger.info(f"ZIP file extracted successfully to {unzip_path}")

        except zipfile.BadZipFile as e:
            logger.error(f"Invalid ZIP file {self.config.local_data_file}: {e}")
            raise
        except FileNotFoundError as e:
            logger.error(f"File not found: {e}")
            raise
        except Exception as e:
            logger.error(f"Error extracting ZIP file: {e}")
            raise


    def perform_data_ingestion(self) -> None:
        """
        Complete data ingestion process: download and extract.
        
        This method orchestrates the full data ingestion pipeline.
        """

        try:
            logger.info("Starting data ingestion process")
            self.download_file()
            self.extract_zip_file()
            logger.info("Data ingestion process completed successfully") 
        except Exception as e:
            logger.error(f"Data ingestion process failed: {e}")
            raise          




In [8]:
try:
    config= ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.perform_data_ingestion()
except Exception as e:
    raise e

[2025-08-01 21:47:06,906: INFO: common: YAML file: config/config.yaml loaded successfully]
[2025-08-01 21:47:06,908: INFO: common: YAML file: params.yaml loaded successfully]
[2025-08-01 21:47:06,909: INFO: common: YAML file: schema.yaml loaded successfully]
[2025-08-01 21:47:06,910: INFO: common: Created directory at artifacts]
[2025-08-01 21:47:06,911: INFO: 3137079986: ConfigurationManager initialized successfully]
[2025-08-01 21:47:06,911: INFO: common: Created directory at artifacts/data_ingestion]
[2025-08-01 21:47:06,912: INFO: 2635355839: Starting data ingestion process]
[2025-08-01 21:47:07,786: INFO: 2635355839: artifacts/data_ingestion/data.zip download completed!]
[2025-08-01 21:47:07,793: INFO: 2635355839: ZIP file extracted successfully to artifacts/data_ingestion]
[2025-08-01 21:47:07,794: INFO: 2635355839: Data ingestion process completed successfully]
